In [1]:
import numpy as np
import pandas as pd
import warnings

class AHP:
    def __init__(self, criteria, factors):
        self.RI = (0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49)
        self.criteria = criteria               #准则
        self.factors = factors                 #因素
        self.num_criteria = criteria.shape[0]
        self.num_factors = factors[0].shape[0]
    def cal_weights(self, input_matrix):
        input_matrix = np.array(input_matrix)
        n, n1 = input_matrix.shape
        eigenvalues, eigenvectors = np.linalg.eig(input_matrix)
        max_idx = np.argmax(eigenvalues)
        max_eigen = eigenvalues[max_idx].real
        eigen = eigenvectors[:, max_idx].real
        eigen = eigen / eigen.sum()
        if n > 9:
            CR = None
            warnings.warn('无法判断一致性')
        else:
            CI = (max_eigen - n) / (n - 1)
            CR = CI / self.RI[n]
        return max_eigen, CR, eigen
    def run(self):
        max_eigen, CR, criteria_eigen = self.cal_weights(self.criteria)
        print('准则层：最大特征值{:<5f},CR={:<5f},检验{}通过'.format(max_eigen, CR, '' if CR < 0.1 else '不'))
        print('准则层权重={}\n'.format(criteria_eigen))

        max_eigen_list, CR_list, eigen_list = [], [], []
        k = 1
        for i in self.factors:
            max_eigen, CR, eigen = self.cal_weights(i)
            max_eigen_list.append(max_eigen)
            CR_list.append(CR)
            eigen_list.append(eigen)
            print('准则 {} 因素层：最大特征值{:<5f},CR={:<5f},检验{}通过'.format(k,max_eigen, CR, '' if CR < 0.1 else '不'))
            print('因素层权重={}\n'.format(eigen))
            k = k + 1    
        return criteria_eigen ,eigen_list

In [2]:
def weight():
    # 准则重要性矩阵
    criteria = np.array([[1, 7, 5, 7, 5],
                         [1 / 7, 1, 2, 3, 3],
                         [1 / 5, 1 / 2, 1,  2,  3],
                         [1 / 7, 1 / 3, 1 / 2, 1, 3],
                         [1 / 5, 1 / 3, 1 / 3, 1 / 3, 1]])

    # 对每个准则，方案优劣排序
    b1 = np.array([[1, 5], [1 / 5, 1]])
    b2 = np.array([[1, 2, 5], [1 / 2, 1, 2], [1 / 5, 1 / 2, 1]])
    b3 = np.array([[1, 5, 6, 8], [1 / 5, 1 ,2, 7], [1 / 6, 1 / 2, 1 ,4],[1 / 8, 1 / 7, 1 / 4, 1]])
    b4 = np.array([[1, 3, 4], [1 / 3, 1, 1], [1 / 4, 1, 1]])
    b5 = np.array([[1, 4, 5, 5], [1 / 4, 1, 2, 4], [1 /5 , 1 / 2, 1, 2], [1 / 5,1 /4,1 / 2, 1]])

    b = [b1, b2, b3, b4, b5]
    a,c = AHP(criteria, b).run()
    return a,c

In [3]:
#模糊综合评价法(FCE)，输入准则权重、因素权重
def fuzzy_eval(criteria, eigen):
    #量化评语（优秀、    良好、    一般、    较差、   非常差）
    score = [1,0.8,0.6,0.4,0.2]
    
    df = pd.read_excel('FCE.xlsx')
    print('单因素模糊综合评价：{}\n'.format(df))
    #把单因素评价数据，拆解到5个准则中
    v1 = df.iloc[0:2,:].values
    v2 = df.iloc[2:5,:].values
    v3 = df.iloc[5:9,:].values
    v4 = df.iloc[9:12,:].values
    v5 = df.iloc[12:16,:].values
   
    vv = [v1,v2,v3,v4,v5]
   
    val = []
    num = len(eigen)
    for i in range(num):
        v = np.dot(np.array(eigen[i]),vv[i])
        print('准则{} , 矩阵积为：{}'.format(i+1,v))
        val.append(v)
       
    # 目标层
    obj = np.dot(criteria, np.array(val))
    print('目标层模糊综合评价：{}\n'.format(obj))
    #综合评分
    eval = np.dot(np.array(obj),np.array(score).T)
    print('综合评价：{}'.format(eval*100))
criteria, eigen=weight()
fuzzy_eval(criteria, eigen)

准则层：最大特征值5.418198,CR=0.084314,检验通过
准则层权重=[0.58293141 0.16396029 0.11819176 0.08112143 0.05379511]

准则 1 因素层：最大特征值2.000000,CR=0.000000,检验通过
因素层权重=[0.83333333 0.16666667]

准则 2 因素层：最大特征值3.005535,CR=0.003075,检验通过
因素层权重=[0.59537902 0.27635046 0.12827052]

准则 3 因素层：最大特征值4.236855,CR=0.070493,检验通过
因素层权重=[0.63674151 0.20373312 0.11691427 0.04261111]

准则 4 因素层：最大特征值3.009203,CR=0.005113,检验通过
因素层权重=[0.63370792 0.19192062 0.17437146]

准则 5 因素层：最大特征值4.131108,CR=0.039020,检验通过
因素层权重=[0.58810136 0.21808417 0.11971903 0.07409544]

单因素模糊综合评价：     优秀   良好   一般   较差   很差
0   0.1  0.4  0.3  0.2  0.0
1   0.5  0.3  0.2  0.0  0.0
2   0.2  0.3  0.2  0.1  0.2
3   0.0  0.2  0.3  0.3  0.2
4   0.2  0.2  0.3  0.2  0.1
5   0.5  0.3  0.2  0.0  0.0
6   0.1  0.3  0.3  0.2  0.1
7   0.1  0.1  0.4  0.2  0.2
8   0.0  0.1  0.3  0.3  0.3
9   0.2  0.3  0.4  0.1  0.0
10  0.1  0.3  0.5  0.0  0.0
11  0.2  0.5  0.3  0.0  0.0
12  0.3  0.3  0.3  0.1  0.0
13  0.1  0.3  0.3  0.2  0.1
14  0.1  0.1  0.5  0.2  0.1
15  0.2  0.3  0.3  0.1